In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\SIDDHARTH\OneDrive\Documents\Job profiles\Code_notes\Deep Learning\diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
x

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
x = scaler.fit_transform(x)

In [8]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size= 0.2)

#### Selecitng optimizer through keras tuner

In [10]:

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
import keras_tuner

In [12]:
y_test.shape

(154,)

In [26]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(32, activation= 'relu', input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values =['adam','SGD','Adagrad','rmsprop'])
    model.compile(optimizer = optimizer, loss='binary_crossentropy',metrics =['accuracy'] )

    return model

In [27]:
tuner = keras_tuner.RandomSearch(build_model,objective='val_accuracy',max_trials=5, directory = 'kerastuner_storage', project_name = 'finding_optimizer')

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
tuner.search(x_train,y_train,epochs =5, validation_data =(x_test,y_test))

In [16]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [17]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(x_train,y_train, batch_size=32, epochs=100,initial_epoch=6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7321 - loss: 0.5506 - val_accuracy: 0.6818 - val_loss: 0.5689
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7541 - loss: 0.5061 - val_accuracy: 0.7013 - val_loss: 0.5587
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7415 - loss: 0.5128 - val_accuracy: 0.7013 - val_loss: 0.5540
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7490 - loss: 0.5005 - val_accuracy: 0.6948 - val_loss: 0.5480
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7634 - loss: 0.4753 - val_accuracy: 0.6948 - val_loss: 0.5430
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7457 - loss: 0.5028 - val_accuracy: 0.7208 - val_loss: 0.5346
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7797 - loss: 0.4729 - val_accuracy: 0.7078 - val_loss: 0.5294
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7666 - loss: 0.4864 - val_accuracy: 0.720

#### selecting number of nodes through keras tuner


In [28]:
def build_model(hp):
    model = Sequential()

    units = hp.Int("units", min_value = 8, max_value= 128, step = 8)

    model.add(Dense(units = units, activation='relu', input_dim = 8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='adam', loss ='binary_crossentropy', metrics=['accuracy'])

    return model

In [29]:
tuner = keras_tuner.RandomSearch(build_model,
                                objective='val_accuracy', 
                                max_trials= 5,
                                directory = "kerastuner_storage",
                                project_name = "selecting_nodes")

In [30]:
tuner.search(x_train, y_train, epochs = 5, validation_data =(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 10s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 96)             │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 961 (3.75 KB)

 Trainable params: 961 (3.75 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(x_train, y_train, batch_size=32, epochs = 100, initial_epoch= 6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8104 - loss: 0.4324 - val_accuracy: 0.7468 - val_loss: 0.5176
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7870 - loss: 0.4383 - val_accuracy: 0.7597 - val_loss: 0.5112
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7750 - loss: 0.4393 - val_accuracy: 0.7792 - val_loss: 0.5023
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7963 - loss: 0.4378 - val_accuracy: 0.7662 - val_loss: 0.5017
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7854 - loss: 0.4345 - val_accuracy: 0.7403 - val_loss: 0.5139
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7672 - loss: 0.4824 - val_accuracy: 0.7403 - val_loss: 0.5034
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8089 - loss: 0.4133 - val_accuracy: 0.7532 - val_loss: 0.4976
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8080 - loss: 0.4228 - val_accuracy: 0.74

#### Selecting number of layers


In [40]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(96, activation='relu', input_dim = 8))

    for i in range(hp.Int('num_layers', min_value = 1, max_value = 8)):
        model.add(Dense(48,activation='relu'))

    model.add(Dense(1, activation= 'sigmoid'))

    model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

    return model

In [41]:
tuner = keras_tuner.RandomSearch(build_model,
                                 objective= 'val_accuracy',
                                 max_trials= 5,
                                 directory = 'kerastuner_storage',
                                 project_name = 'number of layers')

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
tuner.search(x_train, y_train, epochs = 5, validation_data =(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 15s


In [43]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [44]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 96)             │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         4,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,273 (40.13 KB)

 Trainable params: 10,273 (40.13 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.fit(x_train, y_train, epochs= 100, initial_epoch=5, validation_data=(x_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7941 - loss: 0.4169 - val_accuracy: 0.7338 - val_loss: 0.5126
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7970 - loss: 0.4366 - val_accuracy: 0.7662 - val_loss: 0.4844
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7900 - loss: 0.4067 - val_accuracy: 0.7208 - val_loss: 0.5140
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7788 - loss: 0.4024 - val_accuracy: 0.7403 - val_loss: 0.5197
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8192 - loss: 0.3994 - val_accuracy: 0.7403 - val_loss: 0.4896
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8313 - loss: 0.3706 - val_accuracy: 0.7403 - val_loss: 0.5107
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8125 - loss: 0.3927 - val_accuracy: 0.7468 - val_loss: 0.4776
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8344 - loss: 0.3652 - val_accuracy: 0.746

#### tuning everything together

In [51]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units', min_value = 24, max_value = 128, step = 8)
    model.add(Dense(units = units, activation= 'relu', input_dim = 8))
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 8)):
        units = hp.Int('units', min_value = 8, max_value = 128, step = 8)
        model.add(Dense(units = units, activation= 'relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss ='binary_crossentropy', metrics=['accuracy'])
    return model

In [52]:
tuner = keras_tuner.RandomSearch(build_model,
                                 objective= 'val_accuracy',
                                 max_trials= 5,
                                 directory = 'kerastuner_storage',
                                 project_name = 'all in one')

Reloading Tuner from kerastuner_storage\all in one\tuner0.json


In [53]:
tuner.search(x_train, y_train, epochs = 5, validation_data =(x_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 54s


In [54]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\SIDDHARTH\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [55]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 120)            │        14,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,841 (401.72 KB)

 Trainable params: 102,841 (401.72 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
model.fit(x_train, y_train, epochs= 100, initial_epoch= 5, validation_data=(x_test, y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7915 - loss: 0.4628 - val_accuracy: 0.6818 - val_loss: 0.5449
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8282 - loss: 0.4114 - val_accuracy: 0.7468 - val_loss: 0.4992
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8097 - loss: 0.4102 - val_accuracy: 0.7273 - val_loss: 0.5101
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7819 - loss: 0.4376 - val_accuracy: 0.6753 - val_loss: 0.5487
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7695 - loss: 0.4438 - val_accuracy: 0.7273 - val_loss: 0.5061
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8316 - loss: 0.4153 - val_accuracy: 0.7208 - val_loss: 0.5544
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8300 - loss: 0.3780 - val_accuracy: 0.7273 - val_loss: 0.5034
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8505 - loss: 0.3511 - val_accuracy: 0.766